In [1]:
# Install required packages
%pip install -q transformers datasets sentencepiece
%pip install -q pytorch-lightning wandb
%pip install -q donut-python

# MPS acceleration is available on MacOS 12.3+
%pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

# !huggingface-cli login this shouldh be done from the terminal

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%pip3` not found.


In [2]:
from transformers import DonutProcessor, VisionEncoderDecoderModel

#processor = DonutProcessor.from_pretrained("Jac-Zac/thesis_test_donut", revision = '8c5467cb66685e801ec6ff8de7e7fdd247274ed0')
#model = VisionEncoderDecoderModel.from_pretrained("Jac-Zac/thesis_test_donut", revision = '8c5467cb66685e801ec6ff8de7e7fdd247274ed0')
processor = DonutProcessor.from_pretrained("Jac-Zac/thesis_donut")
model = VisionEncoderDecoderModel.from_pretrained("Jac-Zac/thesis_donut")
#processor = DonutProcessor.from_pretrained("Jac-Zac/thesis_test_donut")
#model = VisionEncoderDecoderModel.from_pretrained("Jac-Zac/thesis_test_donut")

In [ ]:
import re
import json
import torch
from tqdm.auto import tqdm
import numpy as np
import random
from PIL import Image

from donut import JSONParseEvaluator
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

# Check that MPS is available
"""
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
else:
    device = torch.device("mps")
"""

model.eval()
model.to(device)

output_list = []
accs = []

# image_path = "/Users/jaczac/Github/Thesis/donut_example/img_resized"
image_path = "/Users/jaczac/Downloads/img_resized"

dataset = load_dataset(image_path, split="test")


for idx, sample in tqdm(enumerate(dataset), total=len(dataset)):
    # prepare encoder inputs
    pixel_values = processor(sample["image"].convert("RGB"), return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)
    # prepare decoder inputs
    task_prompt = "<s_herbarium>"
    decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
    decoder_input_ids = decoder_input_ids.to(device)
    
    # autoregressively generate sequence
    outputs = model.generate(
            pixel_values,
            decoder_input_ids=decoder_input_ids,
            max_length=model.decoder.config.max_position_embeddings,
            early_stopping=True,
            pad_token_id=processor.tokenizer.pad_token_id,
            eos_token_id=processor.tokenizer.eos_token_id,
            use_cache=True,
            num_beams=1,
            bad_words_ids=[[processor.tokenizer.unk_token_id]],
            return_dict_in_generate=True,
        )

    # turn into JSON
    seq = processor.batch_decode(outputs.sequences)[0]
    seq = seq.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
    seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove first task start token
    seq = processor.token2json(seq)

    ground_truth = json.loads(sample["ground_truth"])
    
    # modify ground_truth to replace " " with "" since I would still count it as a correct prediction
    ground_truth = json.loads(sample["ground_truth"].replace('" "', '""'))
    
    evaluator = JSONParseEvaluator()
    score = evaluator.cal_acc(seq, ground_truth)

    accs.append(score)
    output_list.append(seq)

scores = {"accuracies": accs, "mean_accuracy": np.mean(accs)}
print(scores, f"length : {len(accs)}")

Resolving data files:   0%|          | 0/3993 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/857 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/857 [00:00<?, ?it/s]

  0%|          | 0/856 [00:00<?, ?it/s]

/opt/homebrew/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:866: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [74]:
print("Mean accuracy:", np.mean(accs))

Mean accuracy: 0.7638958523977475


In [75]:
print("Mean accuracy:", np.median(accs))

Mean accuracy: 0.830369809893124


In [83]:
mean_without_worst = np.mean(np.sort(accs)[10:])
print("Mean accuracy (excluding worst 10):", mean_without_worst)

Mean accuracy (excluding worst 10): 0.8158658554664913


In [ ]:
# get indices of worst 5 predictions
worst_idxs = np.argsort(accs)[:10].tolist()

# prepare decoder inputs
task_prompt = "<s_herbarium>"
decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
decoder_input_ids = decoder_input_ids.to(device)

for idx in worst_idxs:
    sample = dataset[idx]

    # prepare encoder inputs
    pixel_values = processor(sample["image"].convert("RGB"), return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    # autoregressively generate sequence
    outputs = model.generate(
        pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length=model.decoder.config.max_position_embeddings,
        early_stopping=True,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,
        num_beams=1,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
    )

    # turn into JSON
    seq = processor.batch_decode(outputs.sequences)[0]
    seq = seq.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
    seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove first task start token
    seq = processor.token2json(seq)
    
    print(f"Ground Truth: {sample['ground_truth']}\n")
    print(f"Prediction: {seq}\n")
    print(f"Score: {accs[idx]}\n")
    display(sample["image"])